In [1]:
import os
os.chdir('../')

# 使用MMRazor对ResNet34进行剪枝

1. 回顾MMCLs
2. 搜索最优剪枝结构
3. 剪枝

## 回顾MMCls

In [2]:
# Prepare config path
work_dir='./prune_example/'
config_path=f"{work_dir}/configs/"
! mkdir -p $config_path

pretrained_path='https://download.openmmlab.com/mmclassification/v0/resnet/resnet34_8xb32_in1k_20210831-f257d4e6.pth'

我们直接使用mmcls的配置文件。（这里使用了mmengine的跨库调用功能，请参考[MMEngine文档](https://mmengine.readthedocs.io/zh_CN/latest/tutorials/config.html#id11)）

In [3]:
! echo "_base_ = ['mmcls::resnet/resnet34_8xb32_in1k.py']" > $config_path/pretrain.py
! cat $config_path/pretrain.py

# Run config
# ! python ./tools/train.py $config_path/pretrain.py

_base_ = ['mmcls::resnet/resnet34_8xb32_in1k.py']


In [4]:
! timeout 10 python ./tools/train.py $config_path/pretrain.py

/home/liukai/Documents/mmlab2/others/max/mmengine/mmengine/config/utils.py:50: UserWarning: There is not `Config` define in {'Name': 'convnext-base_3rdparty_in21k', 'Metadata': {'Training Data': 'ImageNet-21k', 'FLOPs': 15359124480, 'Parameters': 88591464}, 'In Collection': 'ConvNeXt', 'Results': None, 'Weights': 'https://download.openmmlab.com/mmclassification/v0/convnext/convnext-base_3rdparty_in21k_20220124-13b83eec.pth', 'Converted From': {'Weights': 'https://dl.fbaipublicfiles.com/convnext/convnext_base_22k_224.pth', 'Code': 'https://github.com/facebookresearch/ConvNeXt'}}
  warnings.warn(f'There is not `Config` define in {model_cfg}')
/home/liukai/Documents/mmlab2/others/max/mmengine/mmengine/config/utils.py:50: UserWarning: There is not `Config` define in {'Name': 'convnext-large_3rdparty_in21k', 'Metadata': {'Training Data': 'ImageNet-21k', 'FLOPs': 34368026112, 'Parameters': 197767336}, 'In Collection': 'ConvNeXt', 'Results': None, 'Weights': 'https://download.openmmlab.com/mm

## 准备剪枝结构搜索config
1. 生成搜索config
2. 运行搜索config
3. 获得搜索出的最优结构

### 1.生成搜索config
我们提供了一个工具可以帮助我们一条命令行生成搜索算法config。该工具需要我们提供两个参数：config路径与checkpoint路径

In [5]:
! python tools/get_search_config.py -h

usage: get_search_config.py [-h] [--flops-min FLOPS_MIN]
                            [--flops-max FLOPS_MAX] [-o O]
                            config checkpoint

Get the config to search the pruning structure of a model

positional arguments:
  config                config of the model
  checkpoint            checkpoint path of the model

optional arguments:
  -h, --help            show this help message and exit
  --flops-min FLOPS_MIN
                        minimal flops
  --flops-max FLOPS_MAX
                        maximal flops
  -o O                  output path to store the search config.


In [6]:
! python tools/get_search_config.py  $config_path/pretrain.py $pretrained_path -o $config_path/search.py   &> /dev/null 
! cat $config_path/search.py 

model = dict(
    _scope_='mmrazor',
    type='SearchWrapper',
    architecture=dict(
        type='ImageClassifier',
        backbone=dict(
            type='ResNet',
            depth=34,
            num_stages=4,
            out_indices=(3, ),
            style='pytorch'),
        neck=dict(type='GlobalAveragePooling'),
        head=dict(
            type='LinearClsHead',
            num_classes=1000,
            in_channels=512,
            loss=dict(type='CrossEntropyLoss', loss_weight=1.0),
            topk=(1, 5)),
        _scope_='mmcls',
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            'https://download.openmmlab.com/mmclassification/v0/resnet/resnet34_8xb32_in1k_20210831-f257d4e6.pth'
        ),
        data_preprocessor=dict(
            mean=[123.675, 116.28, 103.53],
            std=[58.395, 57.12, 57.375],
            to_rgb=True)),
    mutator_cfg=dict(
        type='ChannelMutator',
        channel_unit_cfg=dict(
            type

对于生成的config，我们可以进一步进行修改，以匹配自己的需求。其中"model"和“train_cfg”是最重要的两个字段。

### 2. 运行搜索config

In [7]:
! timeout 60 python ./tools/train.py $config_path/search.py  --work-dir $work_dir/search/

11/08 14:14:57 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.9.13 (main, Aug 25 2022, 23:26:10) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 1574091845
    GPU 0: NVIDIA GeForce GTX 1660 Ti
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.3, V11.3.58
    GCC: gcc (Ubuntu 11.2.0-19ubuntu1) 11.2.0
    PyTorch: 1.12.1+cu113
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.3
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm

### 3. 获取搜索结果

搜索结果存放在work_dir目录下面，我们可以通过以下命令查看我们最终的搜索结果。搜索结果用一个字典表示，字典的key是每个可搜索通道节点的名字（name），value则是该通道节点保留的通道比例。

In [8]:
!  cat $work_dir/search/best_fix_subnet.json

{
    "backbone.conv1_(0, 64)_64": 1.0,
    "backbone.layer1.0.conv1_(0, 64)_64": 0.3426007989589187,
    "backbone.layer1.1.conv1_(0, 64)_64": 0.2969206924310629,
    "backbone.layer1.2.conv1_(0, 64)_64": 0.04568010652785583,
    "backbone.layer2.0.conv1_(0, 128)_128": 1.0,
    "backbone.layer2.0.conv2_(0, 128)_128": 1.0,
    "backbone.layer2.1.conv1_(0, 128)_128": 0.19414045274338726,
    "backbone.layer2.2.conv1_(0, 128)_128": 1.0,
    "backbone.layer2.3.conv1_(0, 128)_128": 1.0,
    "backbone.layer3.0.conv1_(0, 256)_256": 0.19414045274338726,
    "backbone.layer3.0.conv2_(0, 256)_256": 0.8393719574493509,
    "backbone.layer3.1.conv1_(0, 256)_256": 0.39970093211873853,
    "backbone.layer3.2.conv1_(0, 256)_256": 0.13133030626758552,
    "backbone.layer3.3.conv1_(0, 256)_256": 0.21127049269133322,
    "backbone.layer3.4.conv1_(0, 256)_256": 1.0,
    "backbone.layer3.5.conv1_(0, 256)_256": 0.017130039947945933,
    "backbone.layer4.0.conv1_(0, 512)_512": 0.3340357789849457,
    "back

## 准备剪枝Config

1. 生成剪枝config模板
2. 替换剪枝比例

### 1. 生成剪枝config模板


In [9]:
! python ./tools/get_prune_config.py -h

usage: get_prune_config.py [-h] [-o O] config checkpoint

Get the config to prune a model.

positional arguments:
  config      config of the model
  checkpoint  checkpoint path of the model

optional arguments:
  -h, --help  show this help message and exit
  -o O        output path to store the pruning config.


In [10]:
! python ./tools/get_prune_config.py $config_path/pretrain.py $pretrained_path -o $config_path/prune.py &> /dev/null
! cat $config_path/prune.py

model = dict(
    _scope_='mmrazor',
    type='ItePruneAlgorithm',
    architecture=dict(
        type='ImageClassifier',
        backbone=dict(
            type='ResNet',
            depth=34,
            num_stages=4,
            out_indices=(3, ),
            style='pytorch'),
        neck=dict(type='GlobalAveragePooling'),
        head=dict(
            type='LinearClsHead',
            num_classes=1000,
            in_channels=512,
            loss=dict(type='CrossEntropyLoss', loss_weight=1.0),
            topk=(1, 5)),
        _scope_='mmcls',
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            'https://download.openmmlab.com/mmclassification/v0/resnet/resnet34_8xb32_in1k_20210831-f257d4e6.pth'
        ),
        data_preprocessor=dict(
            mean=[123.675, 116.28, 103.53],
            std=[58.395, 57.12, 57.375],
            to_rgb=True)),
    target_pruning_ratio=dict({
        'backbone.conv1_(0, 64)_64': 1.0,
        'backbone.laye

### 2. 替换剪枝比例

In [11]:
# change prune config
from mmengine import fileio,Config
pruning_subnet=fileio.load(f'{work_dir}/search/best_fix_subnet.json')
pruning_config=Config.fromfile(f'{config_path}/prune.py')
pruning_config['model']['target_pruning_ratio']=pruning_subnet
pruning_config.dump(f'{config_path}/prune.py')
! cat $config_path/prune.py

model = dict(
    _scope_='mmrazor',
    type='ItePruneAlgorithm',
    architecture=dict(
        type='ImageClassifier',
        backbone=dict(
            type='ResNet',
            depth=34,
            num_stages=4,
            out_indices=(3, ),
            style='pytorch'),
        neck=dict(type='GlobalAveragePooling'),
        head=dict(
            type='LinearClsHead',
            num_classes=1000,
            in_channels=512,
            loss=dict(type='CrossEntropyLoss', loss_weight=1.0),
            topk=(1, 5)),
        _scope_='mmcls',
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            'https://download.openmmlab.com/mmclassification/v0/resnet/resnet34_8xb32_in1k_20210831-f257d4e6.pth'
        ),
        data_preprocessor=dict(
            mean=[123.675, 116.28, 103.53],
            std=[58.395, 57.12, 57.375],
            to_rgb=True)),
    target_pruning_ratio=dict({
        'backbone.conv1_(0, 64)_64':
        1.0,
        'backb

## 剪枝

In [12]:
# run your prune config
! timeout 10 python ./tools/train.py $config_path/prune.py --work-dir $work_dir/prune

11/08 14:16:03 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.9.13 (main, Aug 25 2022, 23:26:10) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 1825000398
    GPU 0: NVIDIA GeForce GTX 1660 Ti
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.3, V11.3.58
    GCC: gcc (Ubuntu 11.2.0-19ubuntu1) 11.2.0
    PyTorch: 1.12.1+cu113
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.3
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm

In [13]:
# ! rm -r $work_dir